## Load the meshes

In [2]:
import trimesh

mesh_1 = trimesh.load('data/example/off/tr_reg_082.off', process=False, validate=False)
mesh_2 = trimesh.load('data/example/off/tr_reg_096.off', process=False, validate=False)

mesh_template = trimesh.load('data/template/template.off', process=False, validate=False)

In [3]:
import denoisfm.utils.preprocessing_util as preprocessing_util
import torch

def preprocess_mesh(mesh):
    verts = preprocessing_util.normalize_face_area(
        torch.tensor(mesh.vertices),
        torch.tensor(mesh.faces)
        )
    mesh.vertices = preprocessing_util.center_bbox(verts)
    return mesh


mesh_1 = preprocess_mesh(mesh_1)
mesh_2 = preprocess_mesh(mesh_2)
mesh_template = preprocess_mesh(mesh_template)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Predicted correspondences

In [4]:
import torch

p2p_template_1 = torch.load(
    'results/tr_reg_082_template.pt', weights_only=True
)
p2p_template_2 = torch.load(
    'results/tr_reg_096_template.pt', weights_only=True
)
p2p_pairwise = torch.load(
    'results/tr_reg_082_tr_reg_096.pt', weights_only=True
)

In [5]:
def visualize_correspondence(scene, mesh1, mesh2, p2p):
    
    # copy meshes
    mesh1 = mesh1.copy()
    mesh2 = mesh2.copy()
    
    ##################################################
    # color gradient
    ##################################################
    
    assert p2p.shape == (len(mesh2.vertices),), f'shapes not equal, p2p: {p2p.shape}, mesh2 vertices: {len(mesh2.vertices)}'
    
    verts_1 = torch.tensor(mesh1.vertices)
    
    coords_x_norm = torch.zeros_like(verts_1)
    for i in range(3):
        coords_x_norm[:, i] = (verts_1[:, i] - verts_1[:, i].min()) / (verts_1[:, i].max() - verts_1[:, i].min())

    coords_interpolated = torch.zeros(verts_1.shape[0])
    
    # axes for gradient
    for i in [0, 1]:
        coords_interpolated += coords_x_norm[:, i]
        
    cmap1 = trimesh.visual.color.interpolate(coords_interpolated, 'jet')
    cmap2 = cmap1[p2p].clip(0, 255)

    ##################################################
    # apply color to vertices
    ##################################################

    mesh1.visual.vertex_colors = cmap1[:len(mesh1.vertices)].clip(0, 255)
    mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)].clip(0, 255)
       
    ##################################################
    # add the meshes
    ##################################################
    
    # move mesh2 to the right
    mesh2.vertices += [1, 0, 0]
    
    scene.add_geometry(mesh1)
    scene.add_geometry(mesh2)
    
    return scene
    

## Visualize template-wise correspondences

In [6]:
scene = trimesh.Scene()

In [7]:
scene.geometry.clear()

scene = visualize_correspondence(scene, mesh_1, mesh_template, p2p_template_1[0])

scene.show()

In [8]:
scene.geometry.clear()

scene = visualize_correspondence(scene, mesh_2, mesh_template, p2p_template_2[0])

scene.show()

## Visualize pairwise correspondences

In [9]:
scene.geometry.clear()

scene = visualize_correspondence(scene, mesh_1, mesh_2, p2p_pairwise)

scene.show()

In [10]:
import torch
from denoisfm.utils.shape_util import compute_geodesic_distmat

dist_1 = torch.tensor(
        compute_geodesic_distmat(mesh_1.vertices, mesh_1.faces), dtype=torch.float32    
    )

In [11]:
torch.save(dist_1, 'results/tr_reg_082_dist.pt')

In [14]:
dist_mat = compute_geodesic_distmat(mesh_1.vertices, mesh_1.faces).astype('float32')

In [15]:
import scipy.io as sio

sio.savemat(
    '/home/s94zalek_hpc/DenoisingFunctionalMaps/results/dist_32.mat',
    {'dist': dist_mat}
    )